# Введение в NLP, часть 2


In [ ]:
!pip install -U pip
!pip install transformers==4.45.2 # в этой версии нет проблем с предупреждением 
# "trainer.tokenizer is now deprecated. you should use trainer.processing_class instead."
# в других версиях оно неизбежно появляется и мешает при обучении
!pip install datasets torch seqeval evaluate


## NER c BERT (25 баллов)

1. Взять датасет из предыдущего ДЗ и обучить на нём BERT.
2. Обучить BERT на подготовленном датасете
3. Оценить результат, сравнить с моделью из первого ДЗ

### Подготовка данных (5 баллов)

Подумать о:
1) Как subword токенизация повлияет на BIO раззметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

conll2003 = load_dataset("conll2003")
conll2003

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

...

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  н
The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [4]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в ner_tags отображается в метку NER:

In [5]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [6]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [7]:
# получение NER-тегов для subword токенов
def form_labels(example):
    features = conll2003["train"].features["ner_tags"].feature
    tags_str = []
    for tag in example["ner_tags"]:
        tags_str.append(features.int2str(tag)) # текстовые представления NER-тегов для слов
    labels = []
    for i, word in enumerate(example['tokens']): # получение тэгов для токенов, на которые разбивается каждое отдельное слово
        tag = tags_str[i]
        bert_tokens = bert_tokenizer(word).tokens()[1:-1]
        if tag == 'O' or tag[0] == 'I':
            for _ in range(len(bert_tokens)):
                labels.append(features.str2int(tag))
        else:
            for j in range(len(bert_tokens)):
                labels.append(features.str2int(tag) if j == 0 else features.str2int('I' + tag[1:]))
    return [-100] + labels + [-100]

# основная функция
def preprocess_ner_dataset(example):
    new_dict = {}
    input_ids = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    new_dict['input_ids'] = input_ids
    len_tokenized = len(input_ids)
    
    new_dict['token_type_ids'] = [0] * len_tokenized
    new_dict['attention_mask'] = [1] * len_tokenized
    new_dict['labels'] = form_labels(example)
    return new_dict

Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [9]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [10]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 100:
        break
print("Токенизация верна!")

100it [00:00, 685.91it/s]

Токенизация верна!


In [11]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [12]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Применим нашу функцию к всему датасету:

In [13]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [15]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2) Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860). Дополнительных баллов второй вариант не принесёт.

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.

> Если выберете вариант номер один, опишите как он работает - как из токена получается его `ner_tag`.

In [40]:
from transformers import AutoModelForTokenClassification, BertConfig


config = BertConfig.from_pretrained(BASE_NER_MODEL)
config.num_labels = features.num_classes
model = AutoModelForTokenClassification.from_config(config) 

### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [37]:
import numpy as np
import evaluate

metrics_calculator = evaluate.load("seqeval")

def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    predictions = np.argmax(logits, axis=-1)
    
    true_labels = []
    pred_labels = []
    
    for target_seq, pred_seq in zip(target_labels, predictions): # в цикле отбираются предсказания для неспециальных токенов
        true_seq = []
        pred_seq_filtered = []
        
        for true_label, pred_label in zip(target_seq, pred_seq):
            if true_label != -100:
                true_seq.append(features.int2str(int(true_label))) # с np.int64 была ошибка, поэтому явно привел к int :/
                pred_seq_filtered.append(features.int2str(int(pred_label)))
        
        true_labels.append(true_seq)
        pred_labels.append(pred_seq_filtered)
    
    metrics = metrics_calculator.compute(predictions=pred_labels, references=true_labels)
    
    return {
        "accuracy": metrics["overall_accuracy"],
        "precision": metrics["overall_precision"],
        "recall": metrics["overall_recall"],
        "f1": metrics["overall_f1"]
    }

### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop. Дополнительных баллов на этом пути нет.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [32]:
import torch

print(torch.cuda.is_available())

True


In [33]:
from tqdm.notebook import tqdm
tqdm.pandas() # чтобы наверняка отображался прогресс при обучении

In [41]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    eval_strategy="steps",
    eval_steps=250,
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=7e-5,
    warmup_steps = 300,
    weight_decay=0.01,  # Regularization
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_accumulation_steps=10,
    gradient_accumulation_steps=4,
    save_total_limit=2,
    report_to="none",  # Disable logging to W&B
    run_name="ner_experiment",  # Optional: Set a custom run name
    fp16=True,
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ner_dataset["train"],
    eval_dataset=preprocessed_ner_dataset["validation"],
    tokenizer=bert_tokenizer,
    data_collator=data_collator,
    compute_metrics=calculate_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,0.420419,0.874065,0.366264,0.469202,0.411391
500,0.513300,0.285371,0.910240,0.447575,0.641535,0.527284
750,0.513300,0.258692,0.924266,0.529104,0.714406,0.607948
1000,0.104600,0.265580,0.933729,0.584774,0.734265,0.651048
1250,0.104600,0.260597,0.942765,0.644404,0.766409,0.700131
1500,0.024600,0.267808,0.943928,0.661059,0.767082,0.710135


TrainOutput(global_step=1635, training_loss=0.19723025629644364, metrics={'train_runtime': 1145.3635, 'train_samples_per_second': 183.885, 'train_steps_per_second': 1.427, 'total_flos': 5801844121186500.0, 'train_loss': 0.19723025629644364, 'epoch': 14.89749430523918})

### Обработка результатов Результатов (5 баллов)

Подумать о:
1) Во время подготовки данных мы преобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

In [43]:
# compute metrics on test

results = trainer.predict(preprocessed_ner_dataset["test"])

In [44]:
print(results.metrics)

{'test_loss': 0.43847692012786865, 'test_accuracy': 0.9237169285072722, 'test_precision': 0.6067138897443795, 'test_recall': 0.6975920679886686, 'test_f1': 0.6489869873167518, 'test_runtime': 7.0166, 'test_samples_per_second': 492.122, 'test_steps_per_second': 15.392}


Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [45]:
import torch
from datasets import Dataset

In [46]:
# вспомогательная функция для получения списка именованных сущностей
def form_entities(text, inputs, labels, word_ids):
    entities = []
    current_entity = None

    for i, (label, word_id) in enumerate(zip(labels, word_ids)):
        if word_id is None:  # тут игнорируются специальные токены
            continue
    
        start_char, end_char = inputs["offset_mapping"][0][i]
    
        if label.startswith("B-"):  # данные о начале новой сущности
            if current_entity: 
                entities.append(current_entity)
            current_entity = {
                "class": label[2:],
                "text": text[start_char:end_char],
                "start": start_char,
                "stop": end_char
            }
        elif label.startswith("I-") and current_entity and current_entity["class"] == label[2:]:
            current_entity["text"] += " " + text[start_char:end_char]
            current_entity["stop"] = end_char
        else:
            if current_entity:
                entities.append(current_entity)
            current_entity = None
    
    if current_entity:
        entities.append(current_entity)
        
    return entities
    
# основная функция; здесь формируется основной словарь
def do_ner(text):
    output_dict = {"text": text}
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_offsets_mapping=True)
    dataset = Dataset.from_dict({
        "input_ids": inputs["input_ids"].tolist(),
        "attention_mask": inputs["attention_mask"].tolist(),
        "token_type_ids": inputs["token_type_ids"].tolist() if "token_type_ids" in inputs else None,
        "offset_mapping": inputs["offset_mapping"].tolist()
    })
    preds = trainer.predict(dataset)
    logits = preds.predictions  # Shape: (1, seq_len, num_labels)
    predicted_ids = torch.argmax(torch.tensor(logits), dim=-1).squeeze().tolist()
    labels = [features.int2str(i) for i in predicted_ids]
    word_ids = inputs.word_ids(0)
    
    output_dict["entities"] = form_entities(text, inputs, labels, word_ids)
    return output_dict

In [47]:
text = "I want to buy a car in Germany, please." # пример для тестирования функции
results = do_ner(text)
results

{'text': 'I want to buy a car in Germany, please.',
 'entities': [{'class': 'LOC',
   'text': 'Germany',
   'start': tensor(23),
   'stop': tensor(30)}]}

In [48]:
for entity in results['entities']:
    print(text[entity["start"]:entity["stop"]])
    assert text[entity["start"]:entity["stop"]] == entity["text"]

Germany


## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса?
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

In [49]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Место для изучения датасета:

In [50]:
toxic_chat_dataset["train"][0]

{'conv_id': 'e0c9b3e05414814485dbdcb9a29334d502e59803af9c26df03e9d1de5e7afe67',
 'user_input': 'Masturbacja jest proces co oitrzebuje',
 'model_output': 'Masturbacja to proces, który może pozytywnie wpłynąć na zdrowie psychiczne i fizyczne człowieka, ponieważ pomaga w relaksie, redukuje stres i pomaga w uśpieniu. Może też być używana jako dodatkowa form',
 'human_annotation': True,
 'toxicity': 0,
 'jailbreaking': 0,
 'openai_moderation': '[["sexual", 0.4609803557395935], ["sexual/minors", 0.0012527990620583296], ["harassment", 0.0001862536446424201], ["hate", 0.00015521160094067454], ["violence", 6.580814078915864e-05], ["self-harm", 3.212967567378655e-05], ["violence/graphic", 1.5190824342425913e-05], ["self-harm/instructions", 1.0009921425080393e-05], ["hate/threatening", 4.4459093260229565e-06], ["self-harm/intent", 3.378846486157272e-06], ["harassment/threatening", 1.7095695739044459e-06]]'}

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [51]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [ ]:
# ?t5_tokenizer.__call__

In [52]:
PREFIX = "toxic: "
MAX_LENGTH = 512

# словарь из индексов классов в выбранные лейблы
id2label = {
    0: "no",
    1: "yes",
}


def preprocess_dataset(example):
    new_dict = example
    tokens = t5_tokenizer(
        PREFIX + example["user_input"], 
        max_length=MAX_LENGTH, 
        truncation=True
    )
    new_dict['input_ids'] = tokens['input_ids']
    new_dict['attention_mask'] = tokens['attention_mask']
    new_dict['labels'] = t5_tokenizer(id2label[new_dict['toxicity']])['input_ids']
    return new_dict


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

In [53]:
toxic_chat_dataset["train"][0]

{'user_input': 'Masturbacja jest proces co oitrzebuje',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  6664,
  2905,
  9305,
  1191,
  528,
  7,
  17,
  6345,
  576,
  3,
  32,
  155,
  52,
  776,
  3007,
  1924,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

In [54]:
from transformers import AutoModelForSeq2SeqLM


seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Инициализируем соответствующий задаче `DataCollator`.

In [55]:
from transformers import DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [56]:
accuracy_metric = evaluate.load("accuracy")

label2id = {
    "no": 0,
    "yes": 1
}

def compute_metric(eval_predictions):
    preds, labels = eval_predictions

    # текстовое представление без специальных токенов
    decoded_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # лейблы 0 / 1 / -1
    int_preds = [label2id.get(p.strip(), -1) for p in decoded_preds]  # если сгенерировало не "yes" / "no", то -1
    int_labels = [label2id.get(l.strip(), -1) for l in decoded_labels]

    accuracy = accuracy_metric.compute(predictions=int_preds, references=int_labels)['accuracy']
    return {"accuracy": torch.tensor(accuracy, dtype=torch.double)}

In [57]:
def check_compute_metric():
    import torch

    # два предсказания, где токен 150 обозначает токсичный лейбл, токен 120 - нетоксичный лейбл
    preds = torch.tensor(
        [
            [0, 150, 1],  # правильное предсказание - токсичный пример
            [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
        ],
    )
    labels = torch.tensor(
        [
            [150, 1],
            [150, 1],
        ],
    )
    assert torch.isclose(
        compute_metric((preds, labels))["accuracy"],
        torch.tensor(0.5, dtype=torch.double),  # тип тензора тут можно поправить
    )


check_compute_metric()

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

1. На вход в энкодер подается input_ids - список id токенизированного текста, сконкатенированного с префиксом задачи. И плюс attention_mask.
2. В декодере decoder_input_ids (в данном случае labels) - список id для target последовательности, смещенный вправо. Начало обычно заполняется id <PAD> токена - то есть нулями
3. Столько раз, сколько токенов в labels
4. Как я помню, в слой cross-attention декодера подаются key и value вектора из энкодера 

In [58]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=150,
    save_strategy="epoch",
    learning_rate=5e-5,  # вроде бы стандарт для дообучения T5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps = 100,
    weight_decay=0.01,  # для регуляризации
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,  # прочитал,что для seq2seq модели это важно указать
    eval_accumulation_steps=10,
    gradient_accumulation_steps=2,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=seq2seq_model,
    args=training_args,
    train_dataset=toxic_chat_dataset['train'],
    eval_dataset=toxic_chat_dataset['test'],
    tokenizer=t5_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metric
)

In [33]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
150,No log,0.123073,0.928782
300,No log,0.097998,0.939406
450,No log,0.093541,0.940390
600,1.126000,0.086835,0.942554


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=636, training_loss=0.9069236269537008, metrics={'train_runtime': 358.0697, 'train_samples_per_second': 56.771, 'train_steps_per_second': 1.776, 'total_flos': 1419102668783616.0, 'train_loss': 0.9069236269537008, 'epoch': 4.0})

### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета?

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [59]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


negative


Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [44]:
def is_toxic(
    text: str,
    labels2bool={
        "yes": True,
        "no": False
    },
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prexif=PREFIX,
) -> bool:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Get model
    inputs = tokenizer.encode(prexif + text, return_tensors="pt").to(device)
    model.to(device)
    outputs = model.generate(inputs)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return labels2bool[prediction]


# пример вызова с моделью от авторов датасета
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prexif=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
) # для прохождения ассерта должно вернуть False

# пример вызова с обученной выше моделью
assert not is_toxic(
    text="This is just a text",
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prexif=PREFIX,
    labels2bool={
        "yes": True,
        "no": False
    }
) # для прохождения ассерта должно вернуть False

Fin.

Если остались вопросы или есть комментарии, можно написать их ниже: